In [62]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report

In [63]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [64]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [66]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [67]:
# filling the numeric coloumns

num_col=["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

for col in num_col:

    train[col].fillna(train[col].median(),inplace=True)
    test[col].fillna(train[col].median(),inplace=True)

C:\Users\Akshay\AppData\Local\Temp\ipykernel_21904\1774207282.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].median(),inplace=True)
C:\Users\Akshay\AppData\Local\Temp\ipykernel_21904\1774207282.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [68]:
# fill the categorical missing values with mode

cat_colm=["HomePlanet","CryoSleep","Cabin","Destination","VIP"]
for col in cat_colm:
    train[col].fillna(train[col].mode()[0],inplace=True)
    test[col].fillna(test[col].mode()[0],inplace=True)



C:\Users\Akshay\AppData\Local\Temp\ipykernel_21904\1352681597.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0],inplace=True)
C:\Users\Akshay\AppData\Local\Temp\ipykernel_21904\1352681597.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [69]:
# Split 'Cabin' into deck, number, and side
def split_cabin(df):
    cabin_split = df['Cabin'].str.split('/', expand=True)
    df['Deck'] = cabin_split[0]
    df['CabinNum'] = cabin_split[1]
    df['Side'] = cabin_split[2]
    df.drop(columns=['Cabin'], inplace=True)
    return df

train=split_cabin(train)
test=split_cabin(test)




In [70]:
#Fill newly created columns
train.fillna({'Deck': 'Unknown', 'Side': 'U'}, inplace=True)
test.fillna({'Deck': 'Unknown', 'Side': 'U'}, inplace=True)

In [71]:
label = LabelEncoder()
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

for col in cat_features:
    train[col]=label.fit_transform(train[col])
    test[col]=label.fit_transform(test[col])

In [72]:
# split X and y

X = train.drop(["Transported","PassengerId","Name"],axis=1)

y= train["Transported"].astype(int)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [73]:
# Decision Tree

dt=DecisionTreeClassifier(random_state=42)

dt.fit(X_train,y_train)

dt_pred=dt.predict(X_test)

accuracy_score(y_test,dt_pred)

0.7331799884991375

In [74]:
# Random Forest

rf=RandomForestClassifier(n_estimators=200,random_state=42)

rf.fit(X_train,y_train)

rf_pred=rf.predict(X_test)

accuracy_score(y_test,rf_pred)

0.7964347326049454

In [75]:
# Logistic regression

reg=LogisticRegression(max_iter=1000)

reg.fit(X_train,y_train)

reg_pred=reg.predict(X_test)

accuracy_score(y_test,reg_pred)

c:\Users\Akshay\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7694077055779184

In [76]:
print(classification_report(y_test,dt_pred))

              precision    recall  f1-score   support

           0       0.74      0.71      0.73       861
           1       0.73      0.75      0.74       878

    accuracy                           0.73      1739
   macro avg       0.73      0.73      0.73      1739
weighted avg       0.73      0.73      0.73      1739



In [77]:
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       861
           1       0.81      0.78      0.80       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



In [78]:
print(classification_report(y_test,reg_pred))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76       861
           1       0.75      0.81      0.78       878

    accuracy                           0.77      1739
   macro avg       0.77      0.77      0.77      1739
weighted avg       0.77      0.77      0.77      1739



In [80]:
# predict on test set

final_pred = rf.predict(test.drop(['Name', 'PassengerId'], axis=1))

submission=pd.DataFrame({"PassengerId":test["PassengerId"],"Transported":final_pred.astype(bool)})

submission.to_csv("submission.csv",index=False)

print("succesfully created")

succesfully created
